# <center>Project Proposal: Analysis and Prediction of Flight Delays</center>
#### **Authors:** Longfei Guan, Cheng Zhang, Heidi Lantz, Clare Pan
#### **Group:** 20

## 1. Introduction

In recent years, the air transport industry has grown rapidly. It promotes tourism, generates economic growth, and provides employment opportunities (Air Transport Action Group, 2022). However, there is no denying that flight delays are becoming more frequent. The Federal Aviation Administration (FAA) considers a flight to be delayed if it departs 15 minutes after the scheduled time (Trefis Team, 2016). Delays not only inconvenience passengers but also cost airlines billions of dollars and disrupt airport operations, damaging the airline's reputation (Wang et al., 2019). Because airline industries across the world take up a great part of the global economy, the number of flight delays could have a significant impact on customers’ satisfaction with the airlines. Therefore, it is important to investigate and understand the key factors that can influence flight delays. 

This project will use the 2015 Flight Delays and Cancellations dataset published by the U.S. Department of Transportation (2017) to investigate the possible variables that are associated with flight delays and make predictions about the average departure time delay based on the conclusion. 

The dataset includes 14 unique airlines from the United States, but for this project, we will select Delta Airlines as our subject. Delta Airlines is currently the second-largest airline in the world in terms of total U.S. passenger miles and passenger fleet size, and it is a founding member of the SkyTeam alliance, which has more than 325 destinations in 52 countries on six continents (Wikipedia, 2022). We suppose the study of Delta Airlines will be representative and generalizable for the US airline population. We will examine thirteen potential factors from a total of 31 variables that influence airline delays. 

These 13 variables include `MONTH`, `DAY`, `DAY_OF_WEEK`, `AIRLINE`, `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT`, `SCHEDULED_DEPARTURE`, `DEPARTURE_TIME`, `DEPARTURE_DELAY`, `SCHEDULED_TIME`, `ELAPSED_TIME`, `AIR_TIME`, and `DISTANCE`. After determining which of the 13 factors can affect airplane delay times, we will use these variables to predict delta airline delay time. We mainly selected those variables because the size of the original dataset is too large and factored out the variables that have little to no effect on the final model, such as the airplane’s tail number, whether the flight was cancelled or not, etc. After determining which of the 13 factors can affect airplane delay times, we will use these variables to predict delta airline delay time.

<center><img src="image/flight_delay_image.jpeg"/></center>

**Figure 1** Image source: https://www.nortonrosefulbright.com/fr-ca/nrf-transform/client-stories/managing-flight-delay-claims 

- `MONTH`: month of the flight trips
- `DAY`: day of the Flight Trip
- `DAY_OF_WEEK`: day of week of the flight trip
- `AIRLINE`: airline identifier (in our dataset, only one airline: DL (Delta Airlines))
- `ORIGIN_AIRPORT`: starting airport
- `DESTINATION_AIRPORT`: destination airport
- `SCHEDULED_DEPARTUR`: planned departure time (format HHMM or HMM,  where HH or H = hour and MM = minutes)
- `DEPARTURE_TIME`: time of departure (format HHMM or HMM)
- `DEPARTURE_DELAY`: total delay on departure, in minutes (negative times represent early departures)
- `SCHEDULED_TIME`: planned time amount needed for the flight trip, in minutes
- `ELAPSED_TIME`: total flight duration, including taking off, air time, and landing, in minutes
- `AIR_TIME`: flight duration of the time in air, in minutes 
- `DISTANCE`: distance between two airports, in miles

## 2. Preliminary Results

### Loading relevant libraries

In [ ]:
library (tidyverse)
library (repr)
library (infer)
library (broom)
library (GGally)
library (car)

options(warn=-1)

### Reading & cleaning the datasets

* Since the origianl dataset is too huge(592.4MB), we selected and filtered the variables that we specified in the introduction. Then uploaded the dataset onto Github and loaded the data by read_csv(). 

In [ ]:
flight_data <- read_csv("delta_flights.csv")
flight_data <- flight_data[complete.cases(flight_data), ] %>%
  select(-...1,- replicate,-AIRLINE)
head(flight_data)

**Table 1:** Modified version of the Flight Delay dataset. 

In [ ]:
# total levels of the origin airport
cat("Number of Origin Airport levels =", nrow(count(flight_data, ORIGIN_AIRPORT)))

In [ ]:
# top 3 origin airports with their mean delay time
top_3_Origin_Airport_Counts <- count(flight_data, ORIGIN_AIRPORT) %>% filter(n >300)
top_3_Origin_Airport <- flight_data %>% 
filter (ORIGIN_AIRPORT == "ATL" | ORIGIN_AIRPORT == "DTW" | ORIGIN_AIRPORT == "MSP") %>%
group_by(ORIGIN_AIRPORT) %>%
summarize(mean = mean(DEPARTURE_DELAY))
cbind(top_3_Origin_Airport_Counts,top_3_Origin_Airport[,2])

**Table 2** Number of delayed flights and the average minutes of departure delay for the top 3 *origin airports* with the most data.

In [ ]:
# total levels of the destination airport
cat("Number of Destination Airport levels =", nrow(count(flight_data, DESTINATION_AIRPORT)))

In [ ]:
# top 3 destination airports with their mean delay time
top_3_Destination_Airport_Counts <- count(flight_data, DESTINATION_AIRPORT) %>% filter(n >300)
top_3_Destination_Airport <- flight_data %>% 
filter (DESTINATION_AIRPORT == "ATL" | DESTINATION_AIRPORT == "DTW" | DESTINATION_AIRPORT == "MSP") %>%
group_by(DESTINATION_AIRPORT) %>%
summarize(mean = mean(DEPARTURE_DELAY))
cbind(top_3_Destination_Airport_Counts,top_3_Destination_Airport[,2])
flight_data <- flight_data %>%
  select(-ORIGIN_AIRPORT,-DESTINATION_AIRPORT)

**Table 3** Number of delayed flights and the average minutes of departure delay for the top 3 *destination airports* with the most data.

* We begin by checking for the association of the explanatory variables with the response variables. ggparis() could help us visualize it.
* There are too many distinct values in ORIGIN_AIRPORT and DESTINATION_AIRPORT, so we filter out these two categorical variables.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
flight_pair_plots <- flight_data%>% 
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 10),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )
flight_pair_plots

**Figure 2** Pair plot for explanatory and response variables of `flight_data`

* DEPARTURE_DELAY is weakly correlated with all explanatory variables except for SCHEDULED_DEPARTURE and DEPARTURE_TIME.
* DEPARTURE_DELAY has a correlation coefficient value of 0.083 and 0.106 with SCHEDULED_DEPARTURE and DEPARTURE_TIME, respectively. The correlation coefficient value between DEPARTURE_DELAY and other explanatory variables are < 0.01.

### Exploratory Data Analysis

* We did the similar things here but we only compared explanatory variables.
* This will help us diagnose if there is multicollinearity issue in our model.

* In order to analysis the correlation between DEPARTURE_DELAY and the rest continous variables, we fliter out all categorical variables.

In [ ]:
corr_matrix_flight <- flight_data %>% 
  cor() %>%
  as.data.frame() %>%
  rownames_to_column("var1") %>%
  pivot_longer(-var1, names_to = "var2", values_to = "corr")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
corr_matrix_flight_plot <- corr_matrix_flight %>%
  ggplot(aes(var1, var2)) +
  geom_tile(aes(fill = corr), color = "white") +
  scale_fill_distiller("Correlation Coefficient \n",
    palette =  "PuBu",
    direction = 1, limits = c(-1,1)
  ) +
  labs(x = "Input Variable One", y = "Input Variable Two") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(
      angle = 45, vjust = 1,
      size = 10, hjust = 1
    ),
    axis.text.y = element_text(
      vjust = 1,
      size = 10, hjust = 1
    ),
    legend.title = element_text(size = 10, face = "bold"),
    legend.text = element_text(size = 10),
    legend.key.size = unit(2, "cm")
  ) +
  coord_fixed() +
  geom_text(aes(var1, var2, label = round(corr, 2)), color = "black", size = 6)
corr_matrix_flight_plot

**Figure 3** Correlation tiles between different inputs of `flight_data` variables. The darker values of blue shows that the corresponding input variable 1 and 2 have a strong positive correlation.

* There are several strong positive correlations in the graph, which suggests that there are multicollinearity among the explanatory variables.

### Summary Table

* We applied gather() to convert the dataset into a long format, then used summarise() to obtain summary statistic for all interested variables.

In [ ]:
flight_data_long <- flight_data %>%
  gather(factor_key=TRUE)
flight_data_stats <- flight_data_long %>% group_by(key) %>%
  summarise(mean= mean(value), sd= sd(value), max = max(value),min = min(value))
flight_data_stats

**Table 2:** Summary statistics for all variables in `flight_data` 

* Since variables `MONTH`, `DAY`, and `DAY_OF_WEEK` are discrete variables, we descide to convert them into several binary variable in order for fitting the model.
* We convert variable `MONTH` into two binary variables: `SPRING`(1 <= MONTH <= 4), `SUMMER`(5 <= MONTH <= 8), and `WINTER`(9 <= MONTH <= 12).
* We convert variable `DAY` into three binary variables: `START_OF_MONTH`(1 <= DAY <= 10), `MIDDLE_OF_MONTH`(11 <= DAY <= 20), and `END_OF_MONTH`(21 <= DAY <= 31).
* We convert variable `DAY_OF_WEEK` into two variables: `BUSINESS_DAY`(1 <= DAY_OF_WEEK <= 5), and `HOILDAY`(6 <=  DAY_OF_WEEK <= 7).

In [ ]:
flight_data <- within(flight_data, {   
  flight_day <- NA 
  flight_month <- NA 
  flight_day_of_week <- NA 
  flight_day[DAY <= 10] <- "START_OF_MONTH"
  flight_day[DAY >= 11 & DAY <= 20] <- "MIDDLE_OF_MONTH"
  flight_day[DAY >= 21] <- "END_OF_MONTH"
  flight_month[MONTH <= 4] <- "SPRING"
  flight_month[MONTH >= 5 & MONTH <= 8] <- "SUMMER"
  flight_month[MONTH >= 9] <- "WINTER"
  flight_day_of_week[DAY_OF_WEEK <= 5] <- "BUSINESS_DAY"
  flight_day_of_week[DAY_OF_WEEK > 5] <- "HOILDAY"
   } ) %>% select(-DAY,- MONTH,-DAY_OF_WEEK)

flight_data$flight_day <- as.factor(flight_data$flight_day)
flight_data$flight_month <- as.factor(flight_data$flight_month)
flight_data$flight_day_of_week <- as.factor(flight_data$flight_day_of_week)
 
head(flight_data)

**Table 3** Flight delay data with categorical variables

## 3. Methods and Results

In our analysis, we are predicting average flight delays for Delta’s airline in 2015. In this report, we began by loading our dataset from Kaggle, and tidying the data in order to select the variables and columns that we are investigating. For initial visualizations, we began by checking the correlation of variables, as we want to see which variables would be best to predict departure delay time. We made a correlation matrix plot which shows us that some variables are highly correlated. We could also tell that there exist some high amounts of multicollinearity between certain variables. We believe that this report is trustworthy because we have collected real data from a very large data set. From a sample of over 800,000 individual samples, we took a random sample of 5,000 to make sure we do not have a biased sample. We also believe that since there is so much data, we can generate an accurate model that will allow us to have accurate predictions on delay time in flights. We will also be able to test our model to see how accurately it can predict departure delay.

We plan on making a generative model to determine flight delay time, as well as confidence intervals to determine how accurate our model is. We will be splitting our data set into test and training data, and then using a forward stepwise selection process to determine the most accurate model we can make. From there, we can make our best fit model and then test how well we are able to predict departure delay time.

We hope to achieve an accurate model that correctly predicts flight delays. This way, we can make true evaluations on flights with Delta and how likely your flight is to be delayed. This leads to a large impact because everyone can use this data in their own lives. For example, if we find that one airport location is a lot more likely to be delayed with delta, we may be more cautious and avoid flying there for a layover. Furthermore, you can use these calculations to compare with other airline companies to figure out which airline is the most reliable and has the least flight delays. Many applications are useful with this information and our analysis. Overall, this analysis creates a huge impact because flights are common and understanding the reasoning behind flight delays is extremely helpful.


In [ ]:
set.seed(0)
flight_data$ID <- 1:nrow(flight_data)
training_flight_data <- sample_n(flight_data, size = nrow(flight_data) * 0.7,
  replace = FALSE
)

testing_flight_data <- anti_join(flight_data,
  training_flight_data,
  by = "ID"
)
# I now remove the ID variable
#flight_data <- flight_data %>% select(-"ID")
training_flight_data <- training_flight_data %>% select(-"ID")
testing_flight_data <- testing_flight_data %>% select(-"ID")
model_matrix_X_train <- 
    model.matrix(DEPARTURE_DELAY ~ ., data = training_flight_data)[,-1]
matrix_Y_train <- 
    as.matrix(training_flight_data$DEPARTURE_DELAY, ncol = 1)
model_matrix_X_test <- 
    model.matrix(DEPARTURE_DELAY ~ ., data = testing_flight_data)[,-1]
matrix_Y_test <- 
    as.matrix(testing_flight_data$DEPARTURE_DELAY, ncol = 1)
training_matrix_data <- as.matrix(training_flight_data)
testing_matrix_data <- as.matrix(testing_flight_data)

* Installing the `grpreg` package, so that we can generate Lasso models with grouped penalties.

In [ ]:
library(grpreg)
v.group <- c(1,2,3,4,5,6,7,8,8,9,9)

In [ ]:
fit <- grpreg(model_matrix_X_train, matrix_Y_train, group = v.group, penalty="grLasso")
plot(fit)

**Figure 4** Group lasso model for `model_matrix_X_train` and `matrix_Y_train`

In [ ]:
set.seed(0)
cvfit <- cv.grpreg(model_matrix_X_train, matrix_Y_train, v.group, penalty="grLasso")
plot(cvfit)

**Figure 5**  Group lasso model for `model_matrix_X_train` and `matrix_Y_train` with cross-validation k = 10 

In [ ]:
summary(cvfit)

In [ ]:
cat("Lambda.min = ", cvfit$lambda.min)

* We fit a grLasso-penalized linear regression with the training data and 14 parameters. When lambda equals to 0.6532, at minimum cross-validation error (lambda=0.6532)

In [ ]:
flight_lasso_min_add <- grpreg(model_matrix_X_train, matrix_Y_train, group = v.group, penalty="grLasso", lambda = cvfit$lambda.min)
flight_lasso_min_add$beta

**Table 4** 

In [ ]:
library(mltools)
#OLS FULL REGRESSION & PREDICTION
flight_full_OLS_add <- lm(DEPARTURE_DELAY ~., training_flight_data)
flight_test_pred_full_OLS_add <- predict(flight_full_OLS_add, newdata = testing_flight_data)

flight_test_pred_ridge_min_add <- predict(flight_lasso_min_add,
  X = model_matrix_X_test)

flight_lasso_RMSE <- tibble(
  Model = "Group Lasso Regression with minimum MSE",
  R_MSE = rmse(
    flight_test_pred_ridge_min_add,
    testing_flight_data$DEPARTURE_DELAY
  )
)


flight_RMSE_models <- rbind(
  flight_lasso_RMSE,
  tibble(
    Model = "OLS Full Regression",
    R_MSE = rmse(
    flight_test_pred_full_OLS_add,
    testing_flight_data$DEPARTURE_DELAY
  ))
)
flight_RMSE_models

**Table 5** Summary of Group Lasso Regression with minimum MSE and OLS Full Regression with their corresponding RMSE.

* Group Lasso Regression with minimum error has a smller RMSE (stronger prediction power) than the OLS full regression as we expected. Therefore we take the group lasso regression with minimum MSE as our prediction model. However, Lasso shrinks predictor variables to 0 which makes this method biased.
* We can fix this by applying post-lasso method, which is fitting ordinary least squares on the variables selected by Lasso in the testing dataset.

In [ ]:
Post_Lasso_model <- lm(DEPARTURE_DELAY ~ DEPARTURE_TIME + ELAPSED_TIME + flight_day_of_week + flight_month 
                     + flight_day, testing_flight_data)
Post_Lasso_model$coefficient

In [ ]:
lasso_vif <-  vif(Post_Lasso_model)
lasso_vif

* By applying Variance Inflation Factor (VIF), we could obseve that all of the selected variables have tiny VIFs, which supports that the group lasso method have solved the multicollinearity problem among the predictor variables in our model.

## 5. Discussion

## 6. References

Air Transport Action Group (2022). Social and economic benefits of Aviation. (n.d.). https://www.atag.org/our-activities/social-and-economic-benefits-of-aviation.html 

Trefis Team (2016), What Is The Impact Of Flight Delays? https://www.trefis.com/stock/dal/articles/375013/what-is-the-impact-of-flight-delays/2016-08-31

U.S. Department of Transportation (2017, February 9). 2015 flight delays and cancellations. Kaggle. Retrieved November 4, 2022, from https://www.kaggle.com/datasets/usdot/flight-delays 

Wang, Y., Zhu, J., & Liu, C. (2019). The analysis of the influence of delay absorbing sequence on flight delay propagation. IOP Conference Series: Materials Science and Engineering, 688(2), 022006. https://doi.org/10.1088/1757-899x/688/2/022006 

Wikipedia (2022). Delta Air Lines. https://en.wikipedia.org/wiki/Delta_Air_Lines
